In [21]:
import pandas as pd

#### Read & inspect data

In [22]:
df_books = pd.read_csv('data/books_metadata.csv')

C:\Users\barla\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
print(len(df_books))

130808


In [23]:
df_books.head()

,Unnamed: 0,title,series,author,description,genres,pages,publisher,firstPublishDate,awards,setting,coverImg,mod_title,book_id,url,cover_image
0,0,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,14796360,https://www.goodreads.com/book/show/14796360-t...,https://images.gr-assets.com/books/1355036953m...
1,1,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,11534111,https://www.goodreads.com/book/show/11534111-t...,https://images.gr-assets.com/books/1328214586m...
2,2,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,15784152,https://www.goodreads.com/book/show/15784152-t...,https://images.gr-assets.com/books/1344000603m...
3,3,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,14289293,https://www.goodreads.com/book/show/14289293-t...,https://images.gr-assets.com/books/1337792923m...
4,4,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,16051061,https://www.goodreads.com/book/show/16051061-t...,https://images.gr-assets.com/books/1363545717m...


### Create dictionary with the item profiles

Set book id as id in dataframe, remove non-named left-most column

In [24]:
cols = list(df_books.columns)
cols[0] = "id"
df_books.columns = cols
df_books = df_books.drop("id", axis=1)
df_books = df_books.set_index("book_id")

In [25]:
df_books.head()

,title,series,author,description,genres,pages,publisher,firstPublishDate,awards,setting,coverImg,mod_title,url,cover_image
book_id,,,,,,,,,,,,,,
14796360,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,https://www.goodreads.com/book/show/14796360-t...,https://images.gr-assets.com/books/1355036953m...
11534111,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,https://www.goodreads.com/book/show/11534111-t...,https://images.gr-assets.com/books/1328214586m...
15784152,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,https://www.goodreads.com/book/show/15784152-t...,https://images.gr-assets.com/books/1344000603m...
14289293,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,https://www.goodreads.com/book/show/14289293-t...,https://images.gr-assets.com/books/1337792923m...
16051061,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374.0,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,the hunger games,https://www.goodreads.com/book/show/16051061-t...,https://images.gr-assets.com/books/1363545717m...


Create the dictionary from the above dataframe

In [26]:
books_map = df_books.to_dict(orient='index')

In [27]:
books_map

{14796360: {'title': 'The Hunger Games',
  'series': 'The Hunger Games #1',
  'author': 'Suzanne Collins',
  'description': "WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.",
  'genres': "['Young Adult', 'F

**Problem**: The lists in some cells are stored as strings that looks like lists; we want actual lists in the item profiles.<br/>

Columns where we have lists in the cells are: genres, awards, setting.<br/>
Formats:<br/> 
genres = "['item1', 'item2', ...]"<br/>
awards = "[\\'item1\\', \\'item2\\', ...]"<br/>
setting = "['item1', 'item2', ...]"<br/>

We create functions to turn string into list for the two formats above.

In [28]:
def string_to_list(s, awards=False):
    if awards:
        s_ = s.strip("[]").split(", ")
        s = []
        for aw in s_:
            s.append(aw[1:-1]) # Chop off the " from front and back, somewhat tideous but works
    else:
        s = s.strip("[]").split("', '")
        # Remove "'" from first and last item
        s[0] = s[0][1:]
        s[-1] = s[-1][:-1]
    return s

Clean the lists in books map:

In [29]:
for book_id, metadata in books_map.items():
    books_map[book_id]["awards"] = string_to_list(books_map[book_id]["awards"], awards=True) 
    books_map[book_id]["genres"] = string_to_list(books_map[book_id]["genres"], awards=False) 
    books_map[book_id]["setting"] = string_to_list(books_map[book_id]["setting"], awards=False) 

In [30]:
# Problem should be fixed now
books_map

{14796360: {'title': 'The Hunger Games',
  'series': 'The Hunger Games #1',
  'author': 'Suzanne Collins',
  'description': "WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.",
  'genres': ['Young Adult',
   

books_map is now our set of item profiles